## Шаг 1. Создание таблиц с перечисленными структурами

![Итоговый вид таблиц](tables.png)

![Примеры загруженных данных](data_tables_examples.png)

## Шаг 2. Подготовка окружения и создание подключения

In [1]:
%pip install "psycopg[binary]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg

from psycopg.rows import TupleRow

In [3]:
connection = psycopg.connect(
    dbname="hw_2",
    user="postgres",
    password="1234qaz",
    host="localhost",
    port="5432",
)

cur = connection.cursor()

def resalt_output(records: list[TupleRow]) -> None:
    """Вывод информации о записях БД."""
    if not records:
        print("Запрос не дал результатов.")
    else:
        for i, row in enumerate(records):
            print(f"Вывод строки #{i+1}: {row}")

## Шаг 3. Выполнение запросов

1. Вывести все уникальные бренды, у которых есть хотя бы один продукт со стандартной стоимостью выше 1500 долларов, и суммарными продажами не менее 1000 единиц.

In [4]:
query_1 = """
    SELECT DISTINCT p.brand
    FROM product p
    WHERE p.standard_cost > 1500
        AND p.product_id IN (
            SELECT oi.product_id 
            FROM order_items oi 
            GROUP BY oi.product_id 
            HAVING SUM(oi.quantity) >= 1000
        );
"""

cur.execute(query_1)

resalt_output(cur.fetchall())

Вывод строки #1: ('Giant Bicycles',)
Вывод строки #2: ('OHM Cycles',)


2. Для каждого дня в диапазоне с 2017-04-01 по 2017-04-09 включительно вывести количество подтвержденных онлайн-заказов и количество уникальных клиентов, совершивших эти заказы.

In [5]:
query_2 = """
    SELECT 
        o.order_date,
        COUNT(o.order_id) AS order_count,
        COUNT(DISTINCT o.customer_id) AS customers_count
    FROM orders o 
    WHERE o.online_order IS TRUE
        AND o.order_status = 'Approved'
        AND o.order_date BETWEEN '2017-04-01' AND '2017-04-09'
    GROUP BY o.order_date;
"""

cur.execute(query_2)

resalt_output(cur.fetchall())

Вывод строки #1: (datetime.date(2017, 4, 1), 37, 37)
Вывод строки #2: (datetime.date(2017, 4, 2), 29, 29)
Вывод строки #3: (datetime.date(2017, 4, 3), 27, 27)
Вывод строки #4: (datetime.date(2017, 4, 4), 32, 32)
Вывод строки #5: (datetime.date(2017, 4, 5), 33, 32)
Вывод строки #6: (datetime.date(2017, 4, 6), 36, 36)
Вывод строки #7: (datetime.date(2017, 4, 7), 24, 24)
Вывод строки #8: (datetime.date(2017, 4, 8), 33, 33)
Вывод строки #9: (datetime.date(2017, 4, 9), 30, 30)


3. Вывести профессии клиентов:  
 - из сферы IT, чья профессия начинается с Senior;
 - из сферы Financial Services, чья профессия начинается с Lead.  

Для обеих групп учитывать только клиентов старше 35 лет. Объединить выборки с помощью UNION ALL.

In [6]:
query_3 = """
    SELECT 
        c.first_name
        ,c.last_name
        ,c.job_title 
    FROM customer c 
    WHERE c.job_industry_category = 'IT'
        AND c.job_title LIKE 'Senior%'
        AND c.dob IS NOT NULL
        AND AGE(CURRENT_DATE, c.dob) > INTERVAL '35 years'

    UNION ALL

    SELECT 
        c2.first_name
        ,c2.last_name
        ,c2.job_title 
    FROM customer c2
    WHERE c2.job_industry_category = 'Financial Services'
        AND c2.job_title LIKE 'Lead%'
        AND c2.dob IS NOT NULL
        AND AGE(CURRENT_DATE, c2.dob) > INTERVAL '35 years';
"""

cur.execute(query_3)

resalt_output(cur.fetchall())

Вывод строки #1: ('Valaria', 'Hugh', 'Senior Sales Associate')
Вывод строки #2: ('Laureen', 'Blower', 'Senior Developer')


4. Вывести бренды, которые были куплены клиентами из сферы Financial Services, но не были куплены клиентами из сферы IT.

In [7]:
query_4 = """
    SELECT DISTINCT p.brand 
    FROM orders o 
    JOIN customer c ON o.customer_id = c.customer_id 
    JOIN order_items oi ON o.order_id = oi.order_id 
    JOIN product p ON oi.product_id = p.product_id 
    WHERE c.job_industry_category = 'Financial Services'

    except

    SELECT DISTINCT p.brand 
    FROM orders o 
    JOIN customer c ON o.customer_id = c.customer_id 
    JOIN order_items oi ON o.order_id = oi.order_id 
    JOIN product p ON oi.product_id = p.product_id 
    WHERE c.job_industry_category = 'IT';
"""

cur.execute(query_4)

resalt_output(cur.fetchall())

Запрос не дал результатов.


5. Вывести 10 клиентов (ID, имя, фамилия), которые совершили наибольшее количество онлайн-заказов (в штуках) брендов Giant Bicycles, Norco Bicycles, Trek Bicycles, при условии, что они активны и имеют оценку имущества (property_valuation) выше среднего среди клиентов из того же штата.

In [8]:
query_5 = """
    SELECT 
        c.customer_id
        ,c.first_name
        ,c.last_name
        ,COUNT(DISTINCT o.order_id) AS orders_count
    FROM customer c 
    JOIN orders o ON c.customer_id = o.customer_id 
    JOIN order_items oi ON o.order_id = oi.order_id 
    JOIN product p ON oi.product_id = p.product_id 
    WHERE o.online_order IS TRUE
        AND p.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
        AND c.deceased_indicator = 'N'
        AND c.property_valuation > (
            SELECT AVG(c2.property_valuation)
            FROM customer c2
            WHERE c2.state = c.state 
        )
    GROUP BY c.customer_id, c.first_name, c.last_name
    ORDER BY orders_count DESC
    LIMIT 10;
"""

cur.execute(query_5)

resalt_output(cur.fetchall())

Вывод строки #1: (353, 'Antonia', 'Cardis', 7)
Вывод строки #2: (3221, 'Brigid', 'Quigley', 7)
Вывод строки #3: (1302, 'Ericka', 'Eggers', 6)
Вывод строки #4: (534, 'Madel', 'Palffrey', 6)
Вывод строки #5: (787, 'Norma', 'Batrim', 6)
Вывод строки #6: (1117, 'Georgena', 'Guilaem', 6)
Вывод строки #7: (25, 'Geoff', 'Assaf', 6)
Вывод строки #8: (1, 'Laraine', 'Medendorp', 6)
Вывод строки #9: (478, 'Darcey', 'Harberer', 6)
Вывод строки #10: (714, 'Burtie', 'Scintsbury', 6)


6. Вывести всех клиентов (ID, имя, фамилия), у которых нет подтвержденных онлайн-заказов за последний год, но при этом они владеют автомобилем и их сегмент благосостояния не Mass Customer.

In [9]:
query_6 = """
    WITH last_year_approved_orders AS (
        SELECT DISTINCT customer_id
        FROM orders
        WHERE online_order IS TRUE
            AND order_status = 'Approved'
            AND order_date >= CURRENT_DATE - INTERVAL '1 year'
    )
    SELECT
        c.customer_id
        ,c.first_name
        ,c.last_name
    FROM customer c 
    WHERE c.customer_id NOT IN (
        SELECT customer_id
        FROM last_year_approved_orders
    )
        AND c.owns_car = 'Yes'
        AND c.wealth_segment != 'Mass Customer';
"""

cur.execute(query_6)

resalt_output(cur.fetchall())

Вывод строки #1: (318, 'Libbie', 'Castelin')
Вывод строки #2: (1233, 'Keeley', 'Hawsby')
Вывод строки #3: (2538, 'Jacinta', 'Petrillo')
Вывод строки #4: (3558, 'Torey', 'Sarney')
Вывод строки #5: (5, 'Sheila-kathryn', 'Calton')
Вывод строки #6: (6, 'Curr', 'Duckhouse')
Вывод строки #7: (7, 'Fina', 'Merali')
Вывод строки #8: (9, 'Mala', 'Lind')
Вывод строки #9: (13, 'Gabriele', 'Norcross')
Вывод строки #10: (17, 'Heath', 'Faraday')
Вывод строки #11: (21, 'Mile', 'Cammocke')
Вывод строки #12: (23, 'Olav', 'Polak')
Вывод строки #13: (28, 'Fee', 'Zellmer')
Вывод строки #14: (30, 'Darrick', 'Helleckas')
Вывод строки #15: (31, 'Star', 'Praton')
Вывод строки #16: (33, 'Ernst', 'Hacon')
Вывод строки #17: (35, 'Margaretha', 'Strettle')
Вывод строки #18: (44, 'Neron', 'Verick')
Вывод строки #19: (46, 'Kaila', 'Allin')
Вывод строки #20: (55, 'Linc', 'Jillions')
Вывод строки #21: (58, 'Dalenna', 'Pinnock')
Вывод строки #22: (65, 'Yale', 'Tanser')
Вывод строки #23: (71, 'Hoyt', 'Glavias')
Вывод стр

7. Вывести всех клиентов из сферы 'IT' (ID, имя, фамилия), которые купили 2 из 5 продуктов с самой высокой list_price в продуктовой линейке Road.

In [10]:
query_7 = """
    with top5_products_by_price AS (
        SELECT product_id
        FROM product
        WHERE product_line = 'Road'
        ORDER BY list_price DESC
        LIMIT 5
        ), customer_buyed_products AS (
        SELECT 
            c.customer_id
            ,c.first_name
            ,c.last_name
            ,COUNT(DISTINCT p.product_id) AS top5_count
        FROM customer c
        JOIN orders o ON c.customer_id = o.customer_id 
        JOIN order_items oi ON o.order_id = oi.order_id 
        JOIN product p ON oi.product_id = p.product_id
        WHERE c.job_industry_category = 'IT'
            AND p.product_id IN (
                SELECT product_id
                FROM top5_products_by_price
            )
        GROUP BY c.customer_id, c.first_name, c. last_name
    )
    SELECT 
        c2.customer_id
        ,c2.first_name
        ,c2.last_name
    FROM customer_buyed_products c2
    WHERE top5_count >= 2;
"""

cur.execute(query_7)

resalt_output(cur.fetchall())

Вывод строки #1: (799, 'Harland', 'Spilisy')
Вывод строки #2: (983, 'Shaylyn', 'Riggs')
Вывод строки #3: (1683, 'Brenn', 'Bacon')
Вывод строки #4: (1791, 'Ninon', 'Van Der Hoog')
Вывод строки #5: (1820, 'Yard', 'Teeney')
Вывод строки #6: (1887, 'Kynthia', 'Purcer')
Вывод строки #7: (3406, 'Lucy', 'Lackmann')


8. Вывести клиентов (ID, имя, фамилия, сфера деятельности) из сфер IT или Health, которые совершили не менее 3 подтвержденных заказов в период 2017-01-01 по 2017-03-01, и при этом их общий доход от этих заказов превышает 10 000 долларов.
Разделить вывод на две группы (IT и Health) с помощью UNION.

In [11]:
query_8 = """
    with orders_with_conditions AS (
        SELECT
            o.customer_id
            ,COUNT(o.order_id) AS approved_count
            ,SUM(oi.quantity * oi.item_list_price_at_sale) AS total_sum
        FROM orders o
        JOIN order_items oi ON o.order_id = oi.order_id
        WHERE o.order_status = 'Approved'
            AND o.order_date BETWEEN '2017-01-01' AND '2017-03-01' 
        GROUP BY o.customer_id
        HAVING COUNT(o.order_id) >= 3
            AND SUM(oi.quantity * oi.item_list_price_at_sale) > 10000
    )
    SELECT
        c.customer_id
        ,c.first_name
        ,c.last_name
        ,c.job_industry_category 
    FROM customer c 
    JOIN orders_with_conditions owc ON c.customer_id = owc.customer_id
    WHERE c.job_industry_category = 'IT'

    UNION

    SELECT
        c2.customer_id
        ,c2.first_name
        ,c2.last_name
        ,c2.job_industry_category 
    FROM customer c2
    JOIN orders_with_conditions owc ON c2.customer_id = owc.customer_id
    WHERE c2.job_industry_category = 'Health'
    ORDER BY job_industry_category, customer_id;
"""

cur.execute(query_8)

resalt_output(cur.fetchall())

Вывод строки #1: (167, 'Nathalie', 'Tideswell', 'Health')
Вывод строки #2: (173, 'Ebba', 'Hanselmann', 'Health')
Вывод строки #3: (250, 'Kristofer', '', 'Health')
Вывод строки #4: (394, 'Roanne', 'Cowthard', 'Health')
Вывод строки #5: (424, 'Dennie', 'Eunson', 'Health')
Вывод строки #6: (590, 'Ddene', 'Burleton', 'Health')
Вывод строки #7: (607, 'Adelaida', 'Redmond', 'Health')
Вывод строки #8: (695, 'Kevan', 'Kubal', 'Health')
Вывод строки #9: (723, 'Madelina', 'Marte', 'Health')
Вывод строки #10: (1154, 'Chase', 'Youd', 'Health')
Вывод строки #11: (1291, 'Warner', 'Zuker', 'Health')
Вывод строки #12: (1298, 'Guglielmo', 'Yele', 'Health')
Вывод строки #13: (1389, 'Reina', 'Drever', 'Health')
Вывод строки #14: (1533, 'Sonia', 'Rous', 'Health')
Вывод строки #15: (1555, 'Marcello', 'Spenclay', 'Health')
Вывод строки #16: (1762, 'Gaye', 'Steutly', 'Health')
Вывод строки #17: (1913, 'Tamiko', 'Fergie', 'Health')
Вывод строки #18: (1959, 'Arlen', 'Klossek', 'Health')
Вывод строки #19: (2012

In [12]:
connection.close()